In [1]:
!pip install wandb
import wandb
# Replace with your actual API key
api_key = "8f58df9a66485e9ea9149b8b599cb14eb71832dc"

# Login to Weights & Biases
wandb.login(key=api_key)

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.2/2.2 MB 10.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 207.3/207.3 kB 10.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 266.1/266.1 kB 13.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 62.7/62.7 kB 7.8 MB/s eta 0:00:00


wandb: W&B API key is configured. Use `wandb login --relogin` to force relogin
wandb: WARNING If you're specifying your api key in code, ensure this code is not shared publicly.
wandb: WARNING Consider setting the WANDB_API_KEY environment variable, or running `wandb login` from the command line.
wandb: Appending key for api.wandb.ai to your netrc file: /root/.netrc


True

In [2]:
import gym
import numpy as np
from collections import deque
import matplotlib.pyplot as plt
plt.rcParams['figure.figsize'] = (16, 10)
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
from torch.distributions import Categorical
torch.manual_seed(0)
import base64
import io  # For visualization
from gym.wrappers.monitoring import video_recorder
from IPython.display import HTML
from IPython import display
import glob



device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
print(device)

env = gym.make('Acrobot-v1', new_step_api=True)  # Set new_step_api=True
env.reset(seed=0)  # Use reset instead of seed
print('observation space:', env.observation_space)
print('action space:', env.action_space)


wandb.init(project="acrobot-reinforcebbbbb", config={  # Initialize wandb first
    "hidden_size": 32,  # Default values
    "n_episodes": 1000,
    "max_t": 1000,
    "gamma": 0.99,
    "print_every": 100,
    "lr": 1e-2,
})

config = {
    "hidden_size": wandb.config.hidden_size,  # Access wandb.config after initialization
    "n_episodes": wandb.config.n_episodes,
    "max_t": wandb.config.max_t,
    "gamma": wandb.config.gamma,
    "print_every": wandb.config.print_every,
    "lr": wandb.config.lr,
}

class Policy(nn.Module):
    def __init__(self, state_size=6, action_size=3, hidden_size=config["hidden_size"]):
        super(Policy, self).__init__()
        self.fc1 = nn.Linear(state_size, hidden_size)
        self.fc2 = nn.Linear(hidden_size, action_size)

    def forward(self, state):
        x = F.relu(self.fc1(state))
        x = self.fc2(x)
        # we just consider 1 dimensional probability of action
        return F.softmax(x, dim=1)

    def act(self, state):
        state = torch.from_numpy(state).float().unsqueeze(0).to(device)
        probs = self.forward(state).cpu()
        model = Categorical(probs)
        action = model.sample()
        return action.item(), model.log_prob(action)

class Baseline(nn.Module):
    def __init__(self, state_size, hidden_size=32):
        super(Baseline, self).__init__()
        self.fc1 = nn.Linear(state_size, hidden_size)
        self.fc2 = nn.Linear(hidden_size, 1)

    def forward(self, state):
        x = F.relu(self.fc1(state))
        x = self.fc2(x)
        return x


def reinforce(env, policy, optimizer, baseline, baseline_optimizer, n_episodes=1000, max_t=1000, gamma=0.99, print_every=100):
    scores_deque = deque(maxlen=100)
    scores = []
    for e in range(1, n_episodes):
        saved_log_probs = []
        rewards = []
        states = []
        state = env.reset()
        for t in range(max_t):
            states.append(state)
            action, log_prob = policy.act(state)
            saved_log_probs.append(log_prob)
            state, reward, done, truncated, _ = env.step(action)  # Unpack five values
            rewards.append(reward)
            if done or truncated:  # Check for both done and truncated
                break
        scores_deque.append(sum(rewards))
        scores.append(sum(rewards))
        discounts = [gamma ** i for i in range(len(rewards) + 1)]
        R = sum([a * b for a, b in zip(discounts, rewards)])

        # Calculate the baseline
        baseline_values = []
        for state_in_trajectory in states:
            state_tensor = torch.from_numpy(state_in_trajectory).float().unsqueeze(0).to(device)
            baseline_value = baseline(state_tensor)
            baseline_values.append(baseline_value)
        baseline_values = torch.cat(baseline_values, dim=0).requires_grad_()

        # Calculate the policy loss
        policy_loss = []
        for log_prob, baseline_value in zip(saved_log_probs, baseline_values):
            policy_loss.append(-(log_prob * (R - baseline_value)))
        policy_loss = torch.cat(policy_loss).sum()

        # Update the policy
        optimizer.zero_grad()
        policy_loss.backward(retain_graph=True)
        optimizer.step()

        # Update the baseline
        baseline_optimizer.zero_grad()
        baseline_loss = ((baseline_values - R) ** 2).mean()
        baseline_loss.backward()
        baseline_optimizer.step()

        if e % print_every == 0:
            print('Episode {}\tAverage Score: {:.2f}'.format(e, np.mean(scores_deque)))
        if e % 100 == 0 and np.mean(scores_deque) >= -100:
            break

    return scores



wandb.init(project="acrobot-reinforcebbbbb", config=config)  # Initialize wandb

policy = Policy().to(device)
optimizer = optim.Adam(policy.parameters(), lr=1e-2)
baseline = Baseline(state_size=env.observation_space.shape[0]).to(device)
baseline_optimizer = optim.Adam(baseline.parameters(), lr=1e-2)
scores = reinforce(env, policy, optimizer, baseline, baseline_optimizer, n_episodes=2000)
wandb.finish()  # Finish wandb run

wandb: Currently logged in as: bhavik-160990105023. Use `wandb login --relogin` to force relogin


cpu
observation space: Box([ -1.        -1.        -1.        -1.       -12.566371 -28.274334], [ 1.        1.        1.        1.       12.566371 28.274334], (6,), float32)
action space: Discrete(3)


/usr/local/lib/python3.10/dist-packages/gym/utils/passive_env_checker.py:241: DeprecationWarning: `np.bool8` is a deprecated alias for `np.bool_`.  (Deprecated NumPy 1.24)
  if not isinstance(terminated, (bool, np.bool8)):


Episode 100	Average Score: -449.01
Episode 200	Average Score: -309.61
Episode 300	Average Score: -203.08
Episode 400	Average Score: -132.32
Episode 500	Average Score: -106.03
Episode 600	Average Score: -91.28


In [ ]:
sweep_config = {
    'method': 'grid',
    'metric': {
      'name': 'Average Reward',
      'goal': 'maximize'
    },
    'parameters': {
        'hidden_size': {
            'values': [32,64,128]
        },
        'max_t': {
            'values': [500, 1000, 1500]
        },
        'lr': {
            'values': [1e-5, 1e-3,1e-4]
        }
    }
}

In [ ]:
# import wandb

# def train(config=None):
#     with wandb.init(config=config):
#         config = wandb.config

#         # Define the hyperparameters
#         hidden_size = config.hidden_size
#         n_episodes = config.n_episodes
#         max_t = config.max_t
#         lr = config.lr

#         # Initialize the policy and optimizer
#         policy = Policy(hidden_size=hidden_size).to(device)
#         optimizer = optim.Adam(policy.parameters(), lr=lr)

#         def reinforce(policy, optimizer, n_episodes=n_episodes, max_t=max_t, gamma=1.0, print_every=100):
#             scores_deque = deque(maxlen=100)
#             scores = []

#             for e in range(1, n_episodes+1):
#                 saved_log_probs = []
#                 rewards = []
#                 state = env.reset()

#                 # Collect trajectory
#                 for t in range(max_t):
#                     # Sample the action from current policy
#                     action, log_prob = policy.act(state)
#                     saved_log_probs.append(log_prob)
#                     state, reward, done, _ = env.step(action)
#                     rewards.append(reward)
#                     if done:
#                         break

#                 # Calculate the total regret
#                 scores_deque.append(sum(rewards))
#                 scores.append(sum(rewards))

#                 # Recalculate the total regret applying discounted factor
#                 discounts = [gamma ** i for i in range(len(rewards) + 1)]
#                 R = sum([a * b for a, b in zip(discounts, rewards)])

#                 # Calculate the loss
#                 policy_loss = []
#                 for log_prob in saved_log_probs:
#                     # Minimizing the regret instead of maximizing the reward
#                     policy_loss.append(log_prob * R)
#                 policy_loss = torch.cat(policy_loss).sum()

#                 # Backpropagation
#                 optimizer.zero_grad()
#                 policy_loss.backward()
#                 optimizer.step()

#                 if e % print_every == 0:
#                     print(f'Episode {e}\tAverage Regret: {np.mean(scores_deque):.2f}')

#                 wandb.log({"regret": np.mean(scores_deque)})

#             return scores

#         scores = reinforce(policy, optimizer)
#         wandb.run.summary["best_regret"] = np.min(scores)

# # Run the sweep
# sweep_id = wandb.sweep(sweep_config, entity="bhavik-160990105023", project="assignment2")
# wandb.agent(sweep_id, train, count=20)

In [ ]:
# sweep_config = {
#     'method': 'random',
#     'metric': {
#       'name': 'final_regret',
#       'goal': 'minimize'
#     },
#     'parameters': {
#         'hidden_size':{
#             'values': [1000, 2000, 3000]
#         },
#         'n_episodes': {
#             'values': [0, 0.0005, 0.005]
#         },
#         'max_t': {
#             'values': [500, 1000, 1500]
#         },
#         'lr': {
#             'values': [1e-2, 1e-3, 1e-4]
#         }
#     }
# }

In [ ]:
# Initialize a new sweep
sweep_id = wandb.sweep(sweep_config, entity="bhavik-160990105023", project="acrobot-reinforcebbbbb")

Create sweep with ID: ntoty7hf
Sweep URL: https://wandb.ai/bhavik-160990105023/acrobot-reinforcebbbbb/sweeps/ntoty7hf


In [ ]:
# def train(config=None):
#     # Set default values for hyperparameters
#     default_config = {
#         "hidden_size": 32,
#         "n_episodes": 1000,
#         "max_t": 1000,
#         "gamma": 1.0,
#         "print_every": 100,
#         "lr": 1e-2,
#     }

#     # Initialize a new wandb run
#     run = wandb.init(config=config, reinit=True)

#     # If config is None, use the default values
#     if config is None:
#         run.config.update(default_config, allow_val_change=True)
#         config = run.config

#     # Initialize the environment and seed
#     env = gym.make('Acrobot-v1')
#     env.seed(0)

#     # Create the policy network with the specified hyperparameters
#     policy = Policy(state_size=6, action_size=3, hidden_size=config.hidden_size).to(device)
#     optimizer = optim.Adam(policy.parameters(), lr=config.lr)

#     # Run the REINFORCE algorithm with the specified hyperparameters
#     scores = reinforce(policy, optimizer, n_episodes=config.n_episodes, max_t=config.max_t, gamma=config.gamma, print_every=config.print_every)

#     # Log the final score as a summary metric
#     run.summary["final_score"] = np.mean(scores[-100:])

#     # Finish the wandb run
#     run.finish()

#     # Return any necessary values or metrics
#     return np.mean(scores[-100:])

In [ ]:
# def train(config=None):
#     # Set default values for hyperparameters
#     default_config = {
#         "hidden_size": 32,
#         "n_episodes": 2000,
#         "max_t": 1000,
#         "gamma": 1.0,
#         "print_every": 100,
#         "lr": 1e-2,
#     }

#     # Initialize a new wandb run
#     run = wandb.init(config=config, reinit=True)

#     # If config is None, use the default values
#     if config is None:
#         config = default_config

#     run.config.update(config, allow_val_change=True)

#     # Initialize the environment and seed
#     env = gym.make('Acrobot-v1')
#     env.seed(0)

#     # Create the policy network with the specified hyperparameters
#     policy = Policy(state_size=6, action_size=3, hidden_size=config['hidden_size']).to(device)
#     optimizer = optim.Adam(policy.parameters(), lr=config['lr'])

#     # Run the REINFORCE algorithm with the specified hyperparameters
#     scores = reinforce(policy, optimizer, n_episodes=config['n_episodes'], max_t=config['max_t'], gamma=config['gamma'], print_every=config['print_every'])

#     # Log the final score as a summary metric
#     run.summary["final_score"] = np.mean(scores[-100:])

#     # Finish the wandb run
#     run.finish()

#     # Return any necessary values or metrics
#     return np.mean(scores[-100:])

In [ ]:
def train(config=None):
    # Set default values for hyperparameters
    default_config = {
        "hidden_size": 32,
        "n_episodes": 1000,
        "max_t": 1000,
        "gamma": 1.0,
        "print_every": 100,
        "lr": 1e-2,
        "baseline_lr": 1e-2,
    }

    # Initialize a new wandb run
    run = wandb.init(config=config, reinit=True)

    # If config is None, use the default values
    if config is None:
        run.config.update(default_config, allow_val_change=True)
    config = run.config

    # Initialize the environment and seed
    env = gym.make('Acrobot-v1')
    env.seed(0)

    # Get the state and action sizes for the Acrobot-v1 environment
    state_size = env.observation_space.shape[0]  # 6 dimensions
    action_size = env.action_space.n  # 3 actions

    # Create the policy network with the specified hyperparameters
    policy = Policy(state_size=state_size, action_size=action_size, hidden_size=config.hidden_size).to(device)
    optimizer = optim.Adam(policy.parameters(), lr=config.lr)

    # Create the baseline network with the specified hyperparameters
    baseline = Baseline(state_size=state_size).to(device)
    baseline_optimizer = optim.Adam(baseline.parameters(), lr=config.baseline_lr)

    # Run the REINFORCE algorithm with the specified hyperparameters
    scores = reinforce(env, policy, optimizer, baseline, baseline_optimizer, n_episodes=config.n_episodes, max_t=config.max_t, gamma=config.gamma, print_every=config.print_every)

    # Check if the environment is solved
    if np.mean(scores[-100:]) >= -100:  # Adjust the threshold as needed
        print(f'Environment {env.unwrapped.spec.id} solved in {e - 100:d} episodes!\tAverage Score: {np.mean(scores_deque):.2f}')





    # Log the final score as a summary metric
    run.summary["final_score"] = np.mean(scores[-100:])

    # Finish the wandb run
    run.finish()

    # Return any necessary values or metrics
    return np.mean(scores[-100:])

In [ ]:
sweep_id = wandb.sweep(sweep_config, project="acrobot-reinforcebbbbb")
wandb.agent(sweep_id, train, count=50)

Create sweep with ID: 3glra774
Sweep URL: https://wandb.ai/bhavik-160990105023/acrobot-reinforce/sweeps/3glra774


wandb: Agent Starting Run: f7c2beo7 with config:
wandb: 	hidden_size: 32
wandb: 	lr: 1e-05
wandb: 	max_t: 500


wandb: WARNING Config item 'hidden_size' was locked by 'sweep' (ignored update).
wandb: WARNING Config item 'max_t' was locked by 'sweep' (ignored update).
wandb: WARNING Config item 'lr' was locked by 'sweep' (ignored update).
/usr/local/lib/python3.10/dist-packages/gym/core.py:317: DeprecationWarning: WARN: Initializing wrapper in old step API which returns one bool instead of two. It is recommended to set `new_step_api=True` to use new step API. This will be the default behaviour in future.
  deprecation(
/usr/local/lib/python3.10/dist-packages/gym/wrappers/step_api_compatibility.py:39: DeprecationWarning: WARN: Initializing environment in old step API which returns one bool instead of two. It is recommended to set `new_step_api=True` to use new step API. This will be the default behaviour in future.
  deprecation(
/usr/local/lib/python3.10/dist-packages/gym/core.py:256: DeprecationWarning: WARN: Function `env.seed(seed)` is marked as deprecated and will be removed in the future. Pl

Run f7c2beo7 errored:
Traceback (most recent call last):
  File "/usr/local/lib/python3.10/dist-packages/wandb/agents/pyagent.py", line 308, in _run_job
    self._function()
  File "<ipython-input-9-e488ce5cf39c>", line 38, in train
    scores = reinforce(env, policy, optimizer, baseline, baseline_optimizer, n_episodes=config.n_episodes, max_t=config.max_t, gamma=config.gamma, print_every=config.print_every)
  File "<ipython-input-5-6bda22063399>", line 91, in reinforce
    state, reward, done, truncated, _ = env.step(action)  # Unpack five values
ValueError: not enough values to unpack (expected 5, got 4)

wandb: ERROR Run f7c2beo7 errored:
wandb: ERROR Traceback (most recent call last):
wandb: ERROR   File "/usr/local/lib/python3.10/dist-packages/wandb/agents/pyagent.py", line 308, in _run_job
wandb: ERROR     self._function()
wandb: ERROR   File "<ipython-input-9-e488ce5cf39c>", line 38, in train
wandb: ERROR     scores = reinforce(env, policy, optimizer, baseline, baseline_optimize

wandb: WARNING Config item 'hidden_size' was locked by 'sweep' (ignored update).
wandb: WARNING Config item 'max_t' was locked by 'sweep' (ignored update).
wandb: WARNING Config item 'lr' was locked by 'sweep' (ignored update).


wandb: Ctrl + C detected. Stopping sweep.


In [ ]:
# wandb agent <sweep_id>